In [5]:
import os,sys
import json
import pickle

class Data:

    def __init__ (self,structure, tweet_data, source):
        self.structure = structure
        self.tweet_data = tweet_data
        self.source = source

In [ ]:
# compile train data

path = './Datasets/semeval2017-task8-dataset/rumoureval-data'

# read data

lis = []
folders = os.listdir(path)
for folder in folders:
    print(folder)
    fold = os.listdir(path+'/'+folder)
    for i in fold:
        if (i == '.DS_Store'):
            continue
        temp_source = path + '/' + folder + '/' + i + '/source-tweet/'
        temp_replies = path + '/' + folder + '/' + i + '/replies/'
        temp_struct = path + '/' + folder + '/' + i 
        with open(temp_struct + '/structure.json') as f:
            structure = json.load(f)
        source_file = os.listdir(temp_source)
        source = source_file[0].split('.')[0]
        tweet_data = {}	
        with open(temp_source + source_file[0]) as f:
            tweet_data[source] = (json.load(f))

        reply_file = os.listdir(temp_replies)
        for j in reply_file:
            with open(temp_replies + j) as f:
                tweet_data[j.split('.')[0]] = (json.load(f))

        lis.append(Data(structure, tweet_data, source))


for i in lis:
    print(i.source)


label_path = './Datasets/semeval2017-task8-dataset/traindev/rumoureval-subtaskA-dev.json'

training = {}

with open(label_path,'r') as f:
    labels = json.load(f)

count = 0
for i in lis:
    for key in i.tweet_data.keys():
        if key in labels.keys():
            training[(key, labels[key])] = i.tweet_data[key]
            count+=1
            print(count, " - ", key, ":", labels[key])

with open('dev_labels.pickle', 'wb') as handle:
    pickle.dump(training, handle)

In [ ]:
# compile test data

path = './Datasets/semeval2017-task8-test-data'

lis = []
fold = os.listdir(path)
for i in fold:
    if (i == '.DS_Store'):
        continue
    temp_source = path + '/' + i + '/source-tweet/'
    temp_replies = path + '/' + i + '/replies/'
    temp_struct = path + '/' + i 
    with open(temp_struct + '/structure.json') as f:
        structure = json.load(f)
    source_file = os.listdir(temp_source)
    source = source_file[0].split('.')[0]
    tweet_data = {}	
    with open(temp_source + source_file[0]) as f:
        tweet_data[source] = (json.load(f))

    reply_file = os.listdir(temp_replies)
    for j in reply_file:
        with open(temp_replies + j) as f:
            tweet_data[j.split('.')[0]] = (json.load(f))

    lis.append(Data(structure, tweet_data, source))


for i in lis:
    print(i.source)


label_path = './Datasets/subtaska.json'

training = {}

with open(label_path,'r') as f:
    labels = json.load(f)

count = 0
for i in lis:
    for key in i.tweet_data.keys():
        if key in labels.keys():
            training[(key, labels[key])] = i.tweet_data[key]
            count+=1
            print(count, " - ", key, ":", labels[key])

with open('gold_test_labels.pickle', 'wb') as handle:
    pickle.dump(training, handle)

In [133]:
import json
import nltk
import time
import pickle
from nltk.tokenize import RegexpTokenizer
    # pre-processing by nltk
# t_init = time.time()
    
def preprocess(filename):

    default_stopwords = set(nltk.corpus.stopwords.words('english'))
    #will have to add the following custom 
    custom_stopwords = set(["http", "rt", "co"])
    all_stopwords = default_stopwords | custom_stopwords
    # all_stopwords = default_stopwords
    eng_stemmer = nltk.stem.SnowballStemmer('english')
    # eng_stemmer = nltk.stem.PorterStemmer()

    new_dataset = {}
    #X_preprocessed = []
    list_all_words = []
    #     list_all_strings = []
    print("Started preprocessing!")

    i = 0
    
    with open(filename, 'rb') as handle:
        dataset = pickle.load(handle)
    
    for key in dataset.keys():
        tweet = dataset[key]
        text = tweet['text']
        i+=1
        print("Iteration: ", i, " - ", key[0])
        #tokenization
        # words = nltk.word_tokenize(text)
        words = RegexpTokenizer('\w+').tokenize(text)
        #remove single character words
        words = [word for word in words if len(word) > 1]
        #removing numbers
        words = [word for word in words if not word.isnumeric()]
        #convert to lower case
        words = [word.lower() for word in words]
        #stem the words
        words = [eng_stemmer.stem(word) for word in words]
        #remove stopwords
        words = [word for word in words if word not in all_stopwords]
        #X_preprocessed.append(str(words))
#         for word in words:
#             list_all_words.append(word)
        # f.write("%s\n" % str(words))
        new_dataset[key] = words

    #     print("Writing to file...")

    ## I now wish to count the most frequent words and plot them

#     fdist = nltk.FreqDist(list_all_words)
#     fdist.plot(50,cumulative=False)


    print("Finished preprocessing!")

#     t_end = time.time()
#     print("Finished in ", t_end-t_init, " secs" )

    with open(filename.split(".")[0] + '_preproc.pickle', 'wb') as handle:
        pickle.dump(new_dataset, handle)

In [141]:
import json
import nltk
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.externals import joblib

def vectorize_data(training_file,ngram_size,gram_name):

    with open(training_file, 'rb') as handle:
        dataset = pickle.load(handle)

    text = []

    count_vect = CountVectorizer(ngram_range=(ngram_size, ngram_size))
    tfidf_vect = TfidfVectorizer(ngram_range=(ngram_size, ngram_size))

    for key in dataset.keys():
        text_str = ' '.join(dataset[key])
        text.append(text_str)

    # fitting ngram model

    count_vect.fit(text)
    tfidf_vect.fit(text)

    new_dataset = {}

    print("Count vectorizer...")

    for key in dataset.keys():
        vec = count_vect.transform([' '.join(dataset[key])])
        # print(vec.shape)
        new_dataset[key] = vec

    with open(training_file.split('_preproc_new.')[0]+'_bow_' +gram_name+'_new.pickle', 'wb') as handle:
        pickle.dump(new_dataset, handle)

    for key in new_dataset.keys():
        print(new_dataset[key])
        print(new_dataset[key].shape)
        break

    new_dataset = {}

    print("Tfidf...")

    for key in dataset.keys():
        vec = tfidf_vect.transform([' '.join(dataset[key])])
        # print(vec.shape)
        new_dataset[key] = vec

    with open(training_file.split('_preproc_new.')[0]+'_tfidf_' + gram_name+'_new.pickle', 'wb') as handle:
        pickle.dump(new_dataset, handle)

    for key in new_dataset.keys():
        print(new_dataset[key])
        print(new_dataset[key].shape)
        break

    joblib.dump(count_vect, 'bow_'+gram_name+'_new.pkl')
    joblib.dump(tfidf_vect, 'tfidf_'+gram_name+'_new.pkl')

In [97]:
import pickle
from sklearn.externals import joblib
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn import linear_model
import numpy as np

def run_classification(model_file, train_data_file, disp_string, clf):

    model = joblib.load(model_file)
    with open(train_data_file, 'rb') as handle:
        train = pickle.load(handle)

    with open('test_labels_preproc.pickle', 'rb') as handle:
        test = pickle.load(handle)

    vec_cols = 0
    for key in train.keys():
        vec_cols = train[key].shape[1]
        break
    # print(vec_cols)
    # print(len(test.keys()))
    X_train = np.zeros((len(train.keys()),vec_cols),dtype=float)
    # X_train = []
    y_train = []
    X_test = np.zeros((len(test.keys()),vec_cols),dtype=float)
    # X_test = []
    y_test = []

    # print(X_train.shape)
    # print(X_test.shape)

    mapping = {'comment':0, 'support':1, 'deny':2, 'query':3}

    i = 0
    for key in train.keys():
        # print(X_train[i].shape)
        # print(train[key].shape)
        X_train[i] = train[key].toarray()
        y_train.append(mapping[key[1]])
        i+=1

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    print(X_train.shape, y_train.shape)

    i = 0
    for key in test.keys():
        y_test.append(mapping[key[1]])
        X_test[i] = model.transform([' '.join(test[key])]).toarray()
        i+=1

    X_test = np.array(X_test)
    y_test = np.array(y_test)

    print(X_test.shape, y_test.shape)

    # print(X_test)


    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)

    print(disp_string)

    print("Classification accuracy: ", accuracy_score(y_test, y_pred))

    print("Confusion matrix: ", confusion_matrix(y_test, y_pred))
    target_names = ['comment', 'support', 'deny', 'query']
    print(classification_report(y_test, y_pred, target_names=target_names))


In [114]:
nb1 = MultinomialNB()
run_classification('bow_unigram.pkl', 'training_labels_bow_unigram.pickle', 'For MultinomialNB, with unigram bow model: ', nb1)

nb2 = MultinomialNB()
run_classification('bow_bigram.pkl', 'training_labels_bow_bigram.pickle', 'For MultinomialNB, with bigram bow model: ', nb2)

nb3 = MultinomialNB()
run_classification('bow_trigram.pkl', 'training_labels_bow_trigram.pickle', 'For MultinomialNB, with trigram bow model: ', nb3)

nb4 = MultinomialNB()
run_classification('tfidf_unigram.pkl', 'training_labels_tfidf_unigram.pickle', 'For MultinomialNB, with unigram tfidf model: ', nb4)

nb5 = MultinomialNB()
run_classification('tfidf_bigram.pkl', 'training_labels_tfidf_bigram.pickle', 'For MultinomialNB, with bigram tfidf model: ', nb5)

nb6 = MultinomialNB()
run_classification('tfidf_trigram.pkl', 'training_labels_tfidf_trigram.pickle', 'For MultinomialNB, with trigram tfidf model: ', nb6)

svm1 = linear_model.SGDClassifier()
run_classification('bow_unigram.pkl', 'training_labels_bow_unigram.pickle', 'For SGDClassifier, with unigram bow model: ', svm1)

svm2 = linear_model.SGDClassifier()
run_classification('bow_bigram.pkl', 'training_labels_bow_bigram.pickle', 'For SGDClassifier, with bigram bow model: ', svm2)

svm3 = linear_model.SGDClassifier()
run_classification('bow_trigram.pkl', 'training_labels_bow_trigram.pickle', 'For SGDClassifier, with trigram bow model: ', svm3)

svm4 = linear_model.SGDClassifier()
run_classification('tfidf_unigram.pkl', 'training_labels_tfidf_unigram.pickle', 'For SGDClassifier, with unigram tfidf model: ', svm4)

svm5 = linear_model.SGDClassifier()
run_classification('tfidf_bigram.pkl', 'training_labels_tfidf_bigram.pickle', 'For SGDClassifier, with bigram tfidf model: ', svm5)

svm6 = linear_model.SGDClassifier()
run_classification('tfidf_trigram.pkl', 'training_labels_tfidf_trigram.pickle', 'For SGDClassifier, with trigram tfidf model: ', svm6)

(4238, 6718) (4238,)
(1049, 6718) (1049,)
For MultinomialNB, with unigram bow model: 
Classification accuracy:  0.7054337464251669
Confusion matrix:  [[713  48  11   6]
 [ 69  23   2   0]
 [ 61   5   4   1]
 [ 99   7   0   0]]
             precision    recall  f1-score   support

    comment       0.76      0.92      0.83       778
    support       0.28      0.24      0.26        94
       deny       0.24      0.06      0.09        71
      query       0.00      0.00      0.00       106

avg / total       0.60      0.71      0.64      1049

(4238, 24619) (4238,)
(1049, 24619) (1049,)
For MultinomialNB, with bigram bow model: 
Classification accuracy:  0.7235462345090562
Confusion matrix:  [[746  29   3   0]
 [ 81  13   0   0]
 [ 66   5   0   0]
 [104   2   0   0]]
             precision    recall  f1-score   support

    comment       0.75      0.96      0.84       778
    support       0.27      0.14      0.18        94
       deny       0.00      0.00      0.00        71
      query

/home/aditya/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(4238, 24644) (4238,)
(1049, 24644) (1049,)
For MultinomialNB, with trigram bow model: 
Classification accuracy:  0.7397521448999047
Confusion matrix:  [[771   7   0   0]
 [ 89   5   0   0]
 [ 71   0   0   0]
 [106   0   0   0]]
             precision    recall  f1-score   support

    comment       0.74      0.99      0.85       778
    support       0.42      0.05      0.09        94
       deny       0.00      0.00      0.00        71
      query       0.00      0.00      0.00       106

avg / total       0.59      0.74      0.64      1049

(4238, 6718) (4238,)
(1049, 6718) (1049,)
For MultinomialNB, with unigram tfidf model: 
Classification accuracy:  0.7387988560533841
Confusion matrix:  [[770   8   0   0]
 [ 89   5   0   0]
 [ 71   0   0   0]
 [106   0   0   0]]
             precision    recall  f1-score   support

    comment       0.74      0.99      0.85       778
    support       0.38      0.05      0.09        94
       deny       0.00      0.00      0.00        71
      qu

/home/aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


For SGDClassifier, with unigram bow model: 
Classification accuracy:  0.5938989513822688
Confusion matrix:  [[584 101  47  46]
 [ 66  13  10   5]
 [ 47   9  14   1]
 [ 72  18   4  12]]
             precision    recall  f1-score   support

    comment       0.76      0.75      0.76       778
    support       0.09      0.14      0.11        94
       deny       0.19      0.20      0.19        71
      query       0.19      0.11      0.14       106

avg / total       0.60      0.59      0.60      1049

(4238, 24619) (4238,)
(1049, 24619) (1049,)
For SGDClassifier, with bigram bow model: 
Classification accuracy:  0.7244995233555768
Confusion matrix:  [[747  24   4   3]
 [ 81  11   2   0]
 [ 69   0   1   1]
 [102   3   0   1]]
             precision    recall  f1-score   support

    comment       0.75      0.96      0.84       778
    support       0.29      0.12      0.17        94
       deny       0.14      0.01      0.03        71
      query       0.20      0.01      0.02       106


In [113]:
from sklearn.linear_model import LogisticRegression

lr1 = LogisticRegression()
run_classification('bow_unigram.pkl', 'training_labels_bow_unigram.pickle', 'For LogisticRegression, with unigram model: ', lr1)

(4238, 6718) (4238,)
(1049, 6718) (1049,)
For LogisticRegression, with unigram model: 
Classification accuracy:  0.7197330791229742
Confusion matrix:  [[739  28   6   5]
 [ 79  11   2   2]
 [ 69   0   2   0]
 [ 99   4   0   3]]
             precision    recall  f1-score   support

    comment       0.75      0.95      0.84       778
    support       0.26      0.12      0.16        94
       deny       0.20      0.03      0.05        71
      query       0.30      0.03      0.05       106

avg / total       0.62      0.72      0.64      1049



In [112]:
lr2 = LogisticRegression()
run_classification('bow_bigram.pkl', 'training_labels_bow_bigram.pickle', 'For LogisticRegression, with bigram bow model: ', lr2)

(4238, 24619) (4238,)
(1049, 24619) (1049,)
For LogisticRegression, with bigram bow model: 
Classification accuracy:  0.7387988560533841
Confusion matrix:  [[768  10   0   0]
 [ 87   7   0   0]
 [ 71   0   0   0]
 [106   0   0   0]]
             precision    recall  f1-score   support

    comment       0.74      0.99      0.85       778
    support       0.41      0.07      0.13        94
       deny       0.00      0.00      0.00        71
      query       0.00      0.00      0.00       106

avg / total       0.59      0.74      0.64      1049



/home/aditya/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [111]:
lr3 = LogisticRegression()
run_classification('bow_trigram.pkl', 'training_labels_bow_trigram.pickle', 'For LogisticRegression, with trigram bow model: ', lr3)

(4238, 24644) (4238,)
(1049, 24644) (1049,)
For LogisticRegression, with trigram bow model: 
Classification accuracy:  0.7426120114394662
Confusion matrix:  [[778   0   0   0]
 [ 93   1   0   0]
 [ 71   0   0   0]
 [106   0   0   0]]
             precision    recall  f1-score   support

    comment       0.74      1.00      0.85       778
    support       1.00      0.01      0.02        94
       deny       0.00      0.00      0.00        71
      query       0.00      0.00      0.00       106

avg / total       0.64      0.74      0.63      1049



/home/aditya/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [132]:
lr4 = LogisticRegression()
run_classification('tfidf_unigram.pkl', 'training_labels_tfidf_unigram.pickle', 'For LogisticRegression, with unigram tfidf model: ', lr4)

lr5 = LogisticRegression()
run_classification('tfidf_bigram.pkl', 'training_labels_tfidf_bigram.pickle', 'For LogisticRegression, with bigram tfidf model: ', lr5)

lr6 = LogisticRegression()
run_classification('tfidf_trigram.pkl', 'training_labels_tfidf_trigram.pickle', 'For LogisticRegression, with trigram tfidf model: ', lr6)

(4238, 6718) (4238,)
(1049, 6718) (1049,)
For LogisticRegression, with unigram tfidf model: 
Classification accuracy:  0.7340324118207817
Confusion matrix:  [[762  15   0   1]
 [ 86   8   0   0]
 [ 70   1   0   0]
 [104   2   0   0]]
             precision    recall  f1-score   support

    comment       0.75      0.98      0.85       778
    support       0.31      0.09      0.13        94
       deny       0.00      0.00      0.00        71
      query       0.00      0.00      0.00       106

avg / total       0.58      0.73      0.64      1049



/home/aditya/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(4238, 24619) (4238,)
(1049, 24619) (1049,)
For LogisticRegression, with bigram tfidf model: 
Classification accuracy:  0.7359389895138226
Confusion matrix:  [[766  12   0   0]
 [ 88   6   0   0]
 [ 71   0   0   0]
 [106   0   0   0]]
             precision    recall  f1-score   support

    comment       0.74      0.98      0.85       778
    support       0.33      0.06      0.11        94
       deny       0.00      0.00      0.00        71
      query       0.00      0.00      0.00       106

avg / total       0.58      0.74      0.64      1049

(4238, 24644) (4238,)
(1049, 24644) (1049,)
For LogisticRegression, with trigram tfidf model: 
Classification accuracy:  0.7416587225929456
Confusion matrix:  [[778   0   0   0]
 [ 94   0   0   0]
 [ 71   0   0   0]
 [106   0   0   0]]
             precision    recall  f1-score   support

    comment       0.74      1.00      0.85       778
    support       0.00      0.00      0.00        94
       deny       0.00      0.00      0.00      

In [98]:
import numpy as np

X_train = np.load('train_data_w2v.npy')
y_train = np.load('train_label_w2v.npy')
X_test = np.load('test_data_w2v.npy')
y_test = np.load('test_label_w2v.npy')
        
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4238, 103) (4238,)
(1049, 103) (1049,)


In [100]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print("Classification accuracy: ", accuracy_score(y_test, y_pred))

print("Confusion matrix: ", confusion_matrix(y_test, y_pred))
target_names = ['support', 'query','deny','comment']
print(classification_report(y_test, y_pred, target_names=target_names))

Classification accuracy:  0.771210676835081
Confusion matrix:  [[ 23   3   0  68]
 [  0  35   0  71]
 [  1   2   0  68]
 [  8  18   1 751]]
             precision    recall  f1-score   support

    support       0.72      0.24      0.37        94
      query       0.60      0.33      0.43       106
       deny       0.00      0.00      0.00        71
    comment       0.78      0.97      0.87       778

avg / total       0.71      0.77      0.72      1049



In [101]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print("Classification accuracy: ", accuracy_score(y_test, y_pred))

print("Confusion matrix: ", confusion_matrix(y_test, y_pred))
target_names = ['support', 'query','deny','comment']
print(classification_report(y_test, y_pred, target_names=target_names))


/home/aditya/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Classification accuracy:  0.7130600571973308
Confusion matrix:  [[ 29   4   1  60]
 [  4  24   3  75]
 [  3   4   1  63]
 [ 53  23   8 694]]
             precision    recall  f1-score   support

    support       0.33      0.31      0.32        94
      query       0.44      0.23      0.30       106
       deny       0.08      0.01      0.02        71
    comment       0.78      0.89      0.83       778

avg / total       0.66      0.71      0.68      1049



In [106]:
from xgboost import XGBClassifier

xgb = RandomForestClassifier()

xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

print("Classification accuracy: ", accuracy_score(y_test, y_pred))

print("Confusion matrix: ", confusion_matrix(y_test, y_pred))
target_names = ['support', 'query','deny','comment']
print(classification_report(y_test, y_pred, target_names=target_names))

Classification accuracy:  0.7273593898951383
Confusion matrix:  [[ 30   2   0  62]
 [  3  24   0  79]
 [  4   1   2  64]
 [ 45  21   5 707]]
             precision    recall  f1-score   support

    support       0.37      0.32      0.34        94
      query       0.50      0.23      0.31       106
       deny       0.29      0.03      0.05        71
    comment       0.78      0.91      0.84       778

avg / total       0.68      0.73      0.69      1049



In [4]:
import numpy as np

X_train = np.load('train_data_w2v.npy')
y_train = np.load('train_label_w2v.npy')
X_test = np.load('test_data_w2v.npy')
y_test = np.load('test_label_w2v.npy')
        
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)



(4238, 103) (4238,)
(1049, 103) (1049,)


In [45]:
import re
import string
import nltk
# from nltk.corpus import stopwords

def clean_text(text):
    
    default_stopwords = set(nltk.corpus.stopwords.words('english'))
    #will have to add the following custom 
    custom_stopwords = set(["http", "rt", "co"])
    stops = default_stopwords | custom_stopwords
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
#     stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = nltk.stem.SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    
    return text

In [50]:
import json
import nltk
import time
import pickle
from nltk.tokenize import RegexpTokenizer
import pandas as pd
    # pre-processing by nltk

def preprocess_data(input_file):

    default_stopwords = set(nltk.corpus.stopwords.words('english'))
    #will have to add the following custom 
    custom_stopwords = set(["http", "rt", "co"])
    all_stopwords = default_stopwords | custom_stopwords
    # all_stopwords = default_stopwords
    eng_stemmer = nltk.stem.SnowballStemmer('english')
    # eng_stemmer = nltk.stem.PorterStemmer()

    new_dataset = {}
    #X_preprocessed = []
    list_all_words = []
    #     list_all_strings = []
    print("Started preprocessing " + input_file + " !")

    i = 0

    with open(input_file, 'rb') as handle:
        dataset = pickle.load(handle)

    df = pd.DataFrame({'tweet':[], 'label':[]})
    mapping = {'comment':0, 'support':1, 'deny':2, 'query':3}

    i = 0
    for key in dataset.keys():
    #     print(key, dataset[key])
        df.loc[i] = {'tweet':clean_text(dataset[key]['text']), 'label':mapping[key[1]]}
        i+=1

    #     print("Writing to file...")

    print(df)

    df.to_csv(input_file.split(".")[0]+'_preproc.csv')
    
    return df

In [51]:
train_df = preprocess_data('training_labels.pickle')
test_df = preprocess_data('test_labels.pickle')

Started preprocessing training_labels.pickle !
      label                                              tweet
0       1.0  pari manhunt : charliehebdo attack ak - 47 rif...
1       0.0  rt com hope french get foreign legion ass put ...
2       0.0                 rt com gign got work cut today lol
3       0.0                              rt com better grammar
4       0.0        rt com + islamispeac http : t co lhhixcswt9
5       0.0       rt com cnn msnbc foxnew hide msnbc cnn offic
6       0.0  gargirawat thank repli mam realli appreci twee...
7       0.0  rt com islam barbar heinous religion ca not re...
8       0.0                         rt com crazi jew ! sarcasm
9       0.0  gargirawat mam get noopurtiwari mam contact nu...
10      0.0                   bhavdeepnama it tri noopurtiwari
11      0.0                                    rt com mean rpg
12      0.0  scari ! rt com charliehebdo attack ak - 47 rif...
13      0.0   gargirawat eas weapon brandish threaten scenario
14      

In [78]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

MAXLEN = 10
vocabulary_size = 30000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(train_df['tweet'])
sequences = tokenizer.texts_to_sequences(train_df['tweet'])
X_train = pad_sequences(sequences, maxlen=MAXLEN)
y_train = np.array(train_df['label'].tolist())
print(X_train.shape)

(4238, 5)


In [79]:
model = Sequential()
model.add(Embedding(20000, 100, input_length=MAXLEN))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_split=0.4, epochs=20)

Train on 2542 samples, validate on 1696 samples
Epoch 1/20
2542/2542 [==============================] - 4s 2ms/step - loss: 0.6778 - acc: 0.2266 - val_loss: 0.6642 - val_acc: 0.2229
Epoch 2/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.3535 - acc: 0.5551 - val_loss: 0.9359 - val_acc: 0.3833
Epoch 3/20
2542/2542 [==============================] - 3s 1ms/step - loss: -1.0421 - acc: 0.6452 - val_loss: 1.1408 - val_acc: 0.4180
Epoch 4/20
2542/2542 [==============================] - 3s 1ms/step - loss: -1.8277 - acc: 0.7156 - val_loss: 1.8590 - val_acc: 0.4080
Epoch 5/20
2542/2542 [==============================] - 3s 1ms/step - loss: -2.1988 - acc: 0.7486 - val_loss: 2.0758 - val_acc: 0.4062
Epoch 6/20
2542/2542 [==============================] - 3s 1ms/step - loss: -2.4754 - acc: 0.7624 - val_loss: 2.0816 - val_acc: 0.4175
Epoch 7/20
2542/2542 [==============================] - 3s 1ms/step - loss: -2.6139 - acc: 0.7734 - val_loss: 1.8672 - val_acc: 0.4298
Epoch 8/2

In [80]:
test_sequences = tokenizer.texts_to_sequences(test_df['tweet'])
X_test = pad_sequences(test_sequences, maxlen=MAXLEN)
y_test = np.array(test_df['label'].tolist())
print(X_test.shape)

(1049, 5)


In [81]:
from sklearn.metrics import classification_report

score, acc = model.evaluate(X_test, y_test, batch_size=1)
print('Test score:', score)
print('Test accuracy:', acc)

print(model.summary())

target_names = ['comment', 'support', 'deny', 'query']
y_pred = model.predict_classes(X_test, verbose=1)
print(classification_report(y_test, y_pred, target_names=target_names))

1049/1049 [==============================] - 1s 944us/step
Test score: 2.202318824365104
Test accuracy: 0.5309818875119161
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 5, 100)            2000000   
_________________________________________________________________
lstm_12 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 101       
Total params: 2,080,501
Trainable params: 2,080,501
Non-trainable params: 0
_________________________________________________________________
None
1049/1049 [==============================] - 0s 454us/step
             precision    recall  f1-score   support

    comment       0.77      0.66      0.71       778
    support       0.12      0.48      0.19        94
       deny       0.00      0.00      0.00       

/home/aditya/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [95]:
import json
import pandas

df = pd.DataFrame({'tweet':[], 'id':[]})
i = 0

ids = []
with open('tweets_california_shootout.json','r') as f:
    for line in f:
        data = json.loads(line)
        if data['id'] in ids:
            continue
        ids.append(data['id'])
        df.loc[i] = {'tweet':data['text'], 'id':data['id_str']}
        i+=1
#         if i == 1000:
#             break
print(df)

                        id                                              tweet
0      1060611565945151488  RT @CommonSenseWise: Be it #Californiashooting...
1      1060611559616065536  RT @JamesMelville: More Americans have died fr...
2      1060611556084465664  RT @DBolelli: mass shootings are the canary in...
3      1060611555870556160  RT @MeanwhileinCana: Prayers don't stop mass s...
4      1060611552548610048  RT @invisibleman_17: #CaliforniaShooting #Cali...
5      1060611551479119873  RT @JamesMelville: More Americans have died fr...
6      1060611547771351040  RT @USATODAY: "Please pray if you believe....p...
7      1060611545535664128  RT @AynRandPaulRyan: Hey, @jack - Any chance @...
8      1060611540326465536  RT @sjredmond: Just a reminder that 13 months ...
9      1060611538581667841  RT @AynRandPaulRyan: Hey, @jack - Any chance @...
10     1060611535607783424  RT @NBCDFW: Jason Coffman speaks to the media ...
11     1060611534441852930  RT @AmyMek: Thousands of people line

In [96]:
df.to_csv('tweets_california_shootout.csv')

In [131]:
import pandas as pd
import os
import json
import pickle

def read_new_data(filename):
    df = pd.read_csv(filename)

    df = df[df['label'].notnull()]

    print(df)

    df_wrong_label = df[(df['label'] != 'deny') & (df['label'] != 'query')]

    # print(df_wrong_label)
    
    return df

In [120]:
lasvegas = read_new_data('tweets_lasvegas_shootout_labeled.csv')
# df.to_csv('las_vegas_labeled.csv')
california = read_new_data('tweets_california_shootout_labeled.csv')

     Unnamed: 0                   id  \
35           35  1060610910639710208   
38           38  1060610656364322816   
57           57  1060609258247487488   
63           63  1060608835277254658   
74           74  1060607105407836160   
92           92  1060605006754299904   
97           97  1060604191851212800   
102         102  1060602955454386176   
105         105  1060602434811240454   
107         107  1060602375717707776   
108         108  1060602261489897472   
110         110  1060602072259792897   
115         115  1060601273387569152   
120         120  1060599267394412545   
132         132  1060596374704701443   
153         153  1060591703550246912   
168         168  1060589830646652928   
169         169  1060589539872436224   
172         172  1060589020814684163   
176         176  1060588604626472960   
190         190  1060586450578681857   
208         208  1060583198730665985   
212         212  1060582857251348482   
213         213  1060582828331687938   


In [144]:
newdata = lasvegas.append(california)
newdata = newdata.reset_index()
print(newdata)

newdata.to_csv('newdata.csv')

    index  Unnamed: 0            id  \
0      35          35  1.060611e+18   
1      38          38  1.060611e+18   
2      57          57  1.060609e+18   
3      63          63  1.060609e+18   
4      74          74  1.060607e+18   
5      92          92  1.060605e+18   
6      97          97  1.060604e+18   
7     102         102  1.060603e+18   
8     105         105  1.060602e+18   
9     107         107  1.060602e+18   
10    108         108  1.060602e+18   
11    110         110  1.060602e+18   
12    115         115  1.060601e+18   
13    120         120  1.060599e+18   
14    132         132  1.060596e+18   
15    153         153  1.060592e+18   
16    168         168  1.060590e+18   
17    169         169  1.060590e+18   
18    172         172  1.060589e+18   
19    176         176  1.060589e+18   
20    190         190  1.060586e+18   
21    208         208  1.060583e+18   
22    212         212  1.060583e+18   
23    213         213  1.060583e+18   
24    228         228  1.

In [139]:
with open('training_labels_preproc.pickle', 'rb') as handle:
    dataset = pickle.load(handle)
    
labels = newdata['label'].tolist()
tweets = newdata['tweet'].tolist()
ids = newdata['id'].tolist()

# print(ids)

# print(labels)
# print(len(tweets))

default_stopwords = set(nltk.corpus.stopwords.words('english'))
#will have to add the following custom 
custom_stopwords = set(["http", "rt", "co"])
all_stopwords = default_stopwords | custom_stopwords
# all_stopwords = default_stopwords
eng_stemmer = nltk.stem.SnowballStemmer('english')
    
for i in range(len(ids)):
    text = tweets[i]
    words = RegexpTokenizer('\w+').tokenize(text)
    #remove single character words
    words = [word for word in words if len(word) > 1]
    #removing numbers
    words = [word for word in words if not word.isnumeric()]
    #convert to lower case
    words = [word.lower() for word in words]
    #stem the words
    words = [eng_stemmer.stem(word) for word in words]
    #remove stopwords
    words = [word for word in words if word not in all_stopwords]
    dataset[(ids[i],labels[i])] = words
    
with open('training_labels_preproc_new.pickle', 'wb') as handle:
    pickle.dump(dataset, handle)

In [142]:
vectorize_data('training_labels_preproc_new.pickle',3,'trigram')

Count vectorizer...
  (0, 617)	1
  (0, 1002)	1
  (0, 1703)	1
  (0, 3812)	1
  (0, 8882)	1
  (0, 13124)	1
  (0, 15633)	1
  (0, 16975)	1
  (0, 17819)	1
  (0, 18057)	1
  (0, 18249)	1
  (0, 20691)	1
(1, 25081)
Tfidf...
  (0, 20691)	0.30032806953550634
  (0, 18249)	0.28628763530724693
  (0, 18057)	0.28628763530724693
  (0, 17819)	0.28628763530724693
  (0, 16975)	0.28628763530724693
  (0, 15633)	0.30032806953550634
  (0, 13124)	0.28628763530724693
  (0, 8882)	0.28628763530724693
  (0, 3812)	0.28628763530724693
  (0, 1703)	0.28628763530724693
  (0, 1002)	0.28628763530724693
  (0, 617)	0.28628763530724693
(1, 25081)


In [143]:
nb3 = MultinomialNB()
run_classification('bow_trigram_new.pkl', 'training_labels_bow_trigram_new.pickle', 'For MultinomialNB, with trigram bow model: ', nb3)

nb6 = MultinomialNB()
run_classification('tfidf_trigram_new.pkl', 'training_labels_tfidf_trigram_new.pickle', 'For MultinomialNB, with trigram tfidf model: ', nb6)

svm3 = linear_model.SGDClassifier()
run_classification('bow_trigram_new.pkl', 'training_labels_bow_trigram_new.pickle', 'For SGDClassifier, with trigram bow model: ', svm3)

svm6 = linear_model.SGDClassifier()
run_classification('tfidf_trigram_new.pkl', 'training_labels_tfidf_trigram_new.pickle', 'For SGDClassifier, with trigram tfidf model: ', svm6)

(4298, 25081) (4298,)
(1049, 25081) (1049,)
For MultinomialNB, with trigram bow model: 
Classification accuracy:  0.7397521448999047
Confusion matrix:  [[771   7   0   0]
 [ 89   5   0   0]
 [ 71   0   0   0]
 [106   0   0   0]]
             precision    recall  f1-score   support

    comment       0.74      0.99      0.85       778
    support       0.42      0.05      0.09        94
       deny       0.00      0.00      0.00        71
      query       0.00      0.00      0.00       106

avg / total       0.59      0.74      0.64      1049



/home/aditya/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(4298, 25081) (4298,)
(1049, 25081) (1049,)
For MultinomialNB, with trigram tfidf model: 
Classification accuracy:  0.7426120114394662
Confusion matrix:  [[778   0   0   0]
 [ 93   1   0   0]
 [ 71   0   0   0]
 [106   0   0   0]]
             precision    recall  f1-score   support

    comment       0.74      1.00      0.85       778
    support       1.00      0.01      0.02        94
       deny       0.00      0.00      0.00        71
      query       0.00      0.00      0.00       106

avg / total       0.64      0.74      0.63      1049

(4298, 25081) (4298,)
(1049, 25081) (1049,)


/home/aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


For SGDClassifier, with trigram bow model: 
Classification accuracy:  0.7397521448999047
Confusion matrix:  [[774   4   0   0]
 [ 92   2   0   0]
 [ 71   0   0   0]
 [106   0   0   0]]
             precision    recall  f1-score   support

    comment       0.74      0.99      0.85       778
    support       0.33      0.02      0.04        94
       deny       0.00      0.00      0.00        71
      query       0.00      0.00      0.00       106

avg / total       0.58      0.74      0.63      1049

(4298, 25081) (4298,)
(1049, 25081) (1049,)
For SGDClassifier, with trigram tfidf model: 
Classification accuracy:  0.7416587225929456
Confusion matrix:  [[774   4   0   0]
 [ 90   4   0   0]
 [ 71   0   0   0]
 [106   0   0   0]]
             precision    recall  f1-score   support

    comment       0.74      0.99      0.85       778
    support       0.50      0.04      0.08        94
       deny       0.00      0.00      0.00        71
      query       0.00      0.00      0.00       1